In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

from config import *

In [ ]:
# context = Context('amended-1-no_noise', debug=0, dep_var=dep_var2, mulr=0, load_data=False)
# context = Context('amended-3-7', debug=0, dep_var=dep_var2, mulr=7, load_data=False)
context = Context('amended-3-7', debug=1, dep_var=dep_var2, mulr=7, load_data=False)

# data

In [ ]:
# list existing data files
print('Existing data files:', [o.stem for o in Path(context.fn_feather).parent.glob('*.feather')])

fmtr = DataFormatter(context, cycle_feathers=context.fn_cycles)
cycles = fmtr.cycles # shorten notation
# fmtr.drop_redundent_columns() # already done for cycles
idx = fmtr.beginning_index()

target data format: `[flattened matrix: 30x150, extra cont: 1x2, extra cat: n]`

parameters: cat_names

Use no cat for now

In [ ]:
extra_x_var = ['桩号'] + dep_var1
df_conts, stat_x = fmtr.get_x(normalize=True)
extra_x = fmtr.get_extra_x_task2()
deps = fmtr.get_y()
stat_y = [o.loc[context.dep_var] for o in stat_x]
stat_extra_x = [o.loc[extra_x_var] for o in stat_x]
train_cont = flatten_and_cat(df_conts, [extra_x, deps], sl=context.sl)

In [ ]:
train_cont.to_feather(context.fn_feather)
np.savez_compressed(context.fn_np, idx=idx, stat_x=stat_x, stat_y=stat_y, stat_extra_x=stat_extra_x)
context.fn_feather

In [ ]:
# context = Context('amended-1-no_noise', debug=0, dep_var=dep_var2, mulr=1, gpu_start=9)
context = Context('amended-3-7', debug=0, dep_var=dep_var2, mulr=7, gpu_start=9)
databunch = MultiDeptTabularDataBunch.from_df('tmp', context.cyc_cont, context.dep_var, valid_idx=context.valid_idx_tile, bs=context.bs)

In [ ]:
x, y = ni(databunch.train_dl)
[o.shape for o in x], y.shape, torch.isnan(x[1]).sum()

# model

In [ ]:
hyper_params = {
    'use_extra_x': 1,
    'n_hidden': 400,
    'n_layers': 3,
    'hidden_p': 0.3,
    'input_p' : 0.4,
    'weight_p': 0.5,
    'drops': [0.4] * 4,
    'alpha': 2.,
    'beta': 1.,
    'clip': 0.5,
    'loss_func': MSELossFlat()
}

hyper_params['layers'] = [3*(context.n_cont) + int(hyper_params['use_extra_x']) * 3, 5*context.n_cont, 5, len(context.dep_var)]

In [ ]:
learner = get_new_model(context, databunch, hyper_params)

In [ ]:
learner.lr_find()

learner.recorder.plot()

In [ ]:
learner.fit(20, lr=1e-1)

In [ ]:
learner.fit(20, lr=1e-1)

In [ ]:
lr = 1e-2
time_str = time.strftime("%Y-%m-%d_%H:%M:%S")
save_name = '_'.join([str(hyper_params['n_hidden']),context.exp_name,time_str])
callbacks = [SaveModelCallback(learner, name=save_name , every='improvement', monitor='MAPD', mode='min')]
learner.fit(20, lr, callbacks=callbacks)
learner.recorder.plot_metrics()

In [ ]:
lr = 1e-2
time_str = time.strftime("%Y-%m-%d_%H:%M:%S")
save_name = '_'.join([str(hyper_params['n_hidden']),context.exp_name,time_str])
callbacks = [SaveModelCallback(learner, name=save_name , every='improvement', monitor='MAPD', mode='min')]
learner.fit(20, lr, callbacks=callbacks)
learner.recorder.plot_metrics()